In [1]:
from upath import UPath
from pathlib import Path
import pprint
from getpass import getpass

# run pip install -e . in the root directory to install this package
from stacbuilder import *

In [3]:
# Collection configuration
catalog_version = "v0.2"
collection_config_path = Path("config-collection.json")

# Input Paths
tiff_input_path = UPath("s3://eodata/CLMS/Pan-European/High_Resolution_Layers/Forests/Tree_Cover_Density/Status_Maps/Tree_Cover_Density_2018/")
tiffs_glob = '*/*/*_03035_v020.tif'

# Output Paths
output_path = Path("results")
test_output_path = output_path / "test" / catalog_version
publish_output_path = output_path / "publish" / catalog_version
overwrite = True

In [4]:
# list input files
input_files = list_input_files(
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    max_files=None
)
print(f"Found {len(input_files)} input files. 5 first files:")
for i in input_files[:5]: print(i) 

INFO    | 2024-12-20 16:31:13,361 | Found credentials in environment variables.


Found 1166 input files. 5 first files:
s3://eodata/CLMS/Pan-European/High_Resolution_Layers/Forests/Tree_Cover_Density/Status_Maps/Tree_Cover_Density_2018/TCD_2018_010m_al_03035_v020/DATA/TCD_2018_010m_E50N21_03035_v020.tif
s3://eodata/CLMS/Pan-European/High_Resolution_Layers/Forests/Tree_Cover_Density/Status_Maps/Tree_Cover_Density_2018/TCD_2018_010m_al_03035_v020/DATA/TCD_2018_010m_E51N18_03035_v020.tif
s3://eodata/CLMS/Pan-European/High_Resolution_Layers/Forests/Tree_Cover_Density/Status_Maps/Tree_Cover_Density_2018/TCD_2018_010m_al_03035_v020/DATA/TCD_2018_010m_E51N19_03035_v020.tif
s3://eodata/CLMS/Pan-European/High_Resolution_Layers/Forests/Tree_Cover_Density/Status_Maps/Tree_Cover_Density_2018/TCD_2018_010m_al_03035_v020/DATA/TCD_2018_010m_E51N20_03035_v020.tif
s3://eodata/CLMS/Pan-European/High_Resolution_Layers/Forests/Tree_Cover_Density/Status_Maps/Tree_Cover_Density_2018/TCD_2018_010m_al_03035_v020/DATA/TCD_2018_010m_E51N21_03035_v020.tif


In [5]:
# list meta data
asset_metadata = list_asset_metadata(
    collection_config_path=collection_config_path,
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    max_files=1
)
for k in asset_metadata: 
    pprint.pprint(k.to_dict())

RuntimeError: This event loop is already running

In [6]:
def item_postprocessor(item):
    item.properties["proj:epsg"] = 3035
    return item

In [7]:
# list items
stac_items, failed_files = list_stac_items(
    collection_config_path=collection_config_path,
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    max_files=1,
    item_postprocessor=item_postprocessor
)
print(f"Found {len(stac_items)} STAC items")
if failed_files: print(f"Failed files: {failed_files}")

INFO    | 2024-05-30 14:24:12,861 | PROGRESS: AssetMetadataPipeline.collect_stac_items: START: collect_stac_items
INFO    | 2024-05-30 14:24:13,005 | PROGRESS: AssetMetadataPipeline.group_metadata_by_item_id: START: group_metadata_by_item_id
INFO    | 2024-05-30 14:24:13,019 | PROGRESS: AssetMetadataPipeline.group_metadata_by_item_id: DONE: group_metadata_by_item_id
INFO    | 2024-05-30 14:24:13,033 | PROGRESS: AssetMetadataPipeline.collect_stac_items: Converted 0 of 1 AssetMetadata to STAC Items (0.0%)
INFO    | 2024-05-30 14:24:13,130 | PROGRESS: AssetMetadataPipeline.collect_stac_items: DONE: collect_stac_items


Found 1 STAC items


In [8]:
print("First stac item:")
stac_items[0]

First stac item:


<Item id=TCD_2018_010m_E50N21_03035_v020>

In [9]:
# build collection
build_collection(
    collection_config_path=collection_config_path,
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    output_dir=test_output_path,
    overwrite=overwrite,
    link_items=False,
    item_postprocessor=item_postprocessor,
)

# validate collection
validate_collection(
    collection_file=test_output_path / "collection.json",
)

INFO    | 2024-05-30 14:24:13,246 | PROGRESS: AssetMetadataPipeline.build_collection: START: build_collection


INFO    | 2024-05-30 14:24:13,264 | PROGRESS: AssetMetadataPipeline.collect_stac_items: START: collect_stac_items
INFO    | 2024-05-30 15:05:35,261 | PROGRESS: AssetMetadataPipeline.group_metadata_by_item_id: START: group_metadata_by_item_id
INFO    | 2024-05-30 15:05:35,274 | PROGRESS: AssetMetadataPipeline.group_metadata_by_item_id: DONE: group_metadata_by_item_id
INFO    | 2024-05-30 15:05:35,285 | PROGRESS: AssetMetadataPipeline.collect_stac_items: Converted 0 of 897 AssetMetadata to STAC Items (0.0%)
INFO    | 2024-05-30 15:05:35,562 | PROGRESS: AssetMetadataPipeline.collect_stac_items: DONE: collect_stac_items
INFO    | 2024-05-30 15:05:35,572 | PROGRESS: STACCollectionBuilder.create_empty_collection: START: create_empty_collection
INFO    | 2024-05-30 15:05:35,585 | PROGRESS: STACCollectionBuilder.create_empty_collection: DONE: create_empty_collection
INFO    | 2024-05-30 15:05:35,599 | PROGRESS: STACCollectionBuilder.save_collection: START: Saving collection ...
INFO    | 2024-

In [6]:
auth_settings = AuthSettings(
    enabled=True,
    interactive=False,
    token_url="https://sso.terrascope.be/auth/realms/terrascope/protocol/openid-connect/token",
    authorization_url= "https://sso.terrascope.be/auth/realms/terrascope/protocol/openid-connect/auth",
    client_id="terracatalogueclient",
    username = "victor.verhaert",
    password = getpass(),
)
settings = Settings(
    auth=auth_settings,
    stac_api_url="https://stac.openeo.vito.be/",
    collection_auth_info={
            "_auth": {
                "read": ["anonymous"],
                "write": ["stac-openeo-admin", "stac-openeo-editor"]
            }
        },
    bulk_size=1000,  
)
upload_to_stac_api(
    collection_path=test_output_path / "collection.json",
    settings=settings,
)

INFO    | 2024-05-03 11:37:23,011 | Using login with username victor.verhaert via resource owner password credentials flow


PROGRESS: Uploader.upload_items: START upload of items from start=None to stop=None. (offset=-1, limit=-1)
PROGRESS: Uploader.upload_items_bulk: Uploading items 1 to 897
HTTP response: 200 - OK: body: Successfully added 897 Items.
PROGRESS: Uploader.upload_items: DONE upload of items from start=None to stop=None. (offset=-1, limit=-1)


In [ ]:
bbox = box(
    11.046005504476401,
    47.38783029804821,
    12.411462626880093,
    48.3083796083107
)
bbox.to_geojson()

{'type': 'Polygon',
 'coordinates': (((12.411462626880093, 47.38783029804821),
   (12.411462626880093, 48.3083796083107),
   (11.046005504476401, 48.3083796083107),
   (11.046005504476401, 47.38783029804821),
   (12.411462626880093, 47.38783029804821)),)}